In [58]:
from utils.Variable_setup import *
from utils.Extraction_functions import *
from utils.ICC_functions import *
import os

In [59]:
import pickle


with open(os.path.join(data_path,'SAGAB_Basic_Metric_ROIWISE_Extracted_Data.pckl'), 'rb') as handle:
    Extracted_Data = pickle.load(handle)

In [60]:
preprocs = ['GDT', 'Regular']
paradigms = ['Single_Band', 'Multi_Band', 'Multi_Shell']

## Generate ICCs

In [ ]:
import numpy as np
ICC_Data = {}
ICC_Data['Description'] = ' ##SAGAB### ICC Values calculated from extracted Basic Metric data'

for proc in preprocs:
    ICC_Data[proc] = {}
    for digm in paradigms:
        ICC_Data[proc][digm] = {}
        for met in Metrics:
            ICC_Data[proc][digm][met] = {}
            for ROI in ROIS:
                ICC_Data[proc][digm][met][ROI] = {}

In [ ]:
for ROI in ROIS:
    for proc in preprocs:
        for digm in paradigms:
            for met in Metrics:
                ICC_Data[proc][digm][met][ROI] = ICC(Extracted_Data[proc][digm][met][ROI])

In [ ]:
#Save ICC numpy
with open(os.path.join(data_path,'SAGAB_Basic_Metric_ROIWISE_ICC_Scores.pckl'), 'wb') as handle:
    pickle.dump(Extracted_Data, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Calculate Power

In [69]:
import matplotlib.pyplot as plt
import math
from statsmodels.stats.power import  tt_ind_solve_power
%matplotlib inline

Ns = [1.02, 1.04, 1.06, 1.08, 1.10]

Required_Subs_Data = {}
Required_Subs_Data['Description'] = 'Number of subjects required to detect metric change of 2/4/6/8/10%'

for ROI in ROIS:
    Required_Subs_Data[ROI] = {}
    for proc in preprocs:
        Required_Subs_Data[ROI][proc] = {}
        for digm in paradigms:
            Required_Subs_Data[ROI][proc][digm] = {}
            for met in metrics:
                Required_Subs_Data[ROI][proc][digm][met] = {}
                for digm in paradigms:
                    CVbs = ICC_Data[proc][digm][met][ROI][2]
                    Required_Subs_Data[ROI][proc][digm][met] = []

                    #A population of healthies has a gaussian profile with this mu, sigma 
                    mu = np.nanmean(Extracted_Data[proc][digm][met][ROI])
                    sigma = CVbs * mu
                    for N in Ns:
                        std_diff = (mu*N - mu) / sigma
                        subjects = tt_ind_solve_power(effect_size = std_diff, nobs1 = None, alpha = 0.05, power = 0.80, ratio=1, alternative = 'larger' )
                        Required_Subs_Data[ROI][proc][digm][met].append(math.ceil(subjects))

684.0563897349399 0.06747119793788978 46.1541040724844


NameError: name 'ceil' is not defined

In [ ]:
import pickle


with open(os.path.join(data_path,'SAGAB_Basic_Metric_ROIWISE_Power_Subject_Counts.pckl'), 'wb') as handle:
    pickle.dump(Extracted_Data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [74]:
import matplotlib.pyplot as plt
import math
from statsmodels.stats.power import  tt_ind_solve_power
%matplotlib inline

Ns = [1.01, 1.03, 1.06, 1.1, 1.15, 1.20]


proc = 'Regular'
metrics = ['Radial', 'Axial', 'Trace']
ROI_Names = ['Left_Caudate', 'Right_Caudate']

N_subs = {}
N_subs['Percentages'] = Ns

for met in metrics:
    N_subs[met] = {}
    for digm in paradigms:
        N_subs[met][digm] = {}
        for roi_i, ROI in enumerate(Caudate_ROIS):
            N_subs[met][digm][ROI_Names[roi_i]]= []

for met in metrics:
    for digm in paradigms:
        for roi_i, ROI in enumerate(Caudate_ROIS):
            CVbs = ICC_Data[proc][digm][met][ROI][2]

            #A population of healthies has a gaussian profile with this mu, sigma 
            mu = np.nanmean(Extracted_Data[proc][digm][met][ROI])
            sigma = CVbs * mu
            for N in Ns:
                std_diff = (mu*N - mu) / sigma
                N_subs[met][digm][ROI_Names[roi_i]].append(math.ceil(tt_ind_solve_power(effect_size = std_diff, nobs1 = None, alpha = 0.05, power = 0.80, ratio=1, alternative = 'larger' )))
                

In [76]:
Advanced_Metrics

['DKI', 'DKI_Thresholded', 'NODDI', 'MAPMRI']

In [40]:
Caudate_Data[proc][digm][met][ROI]

array([[1339.564996, 1316.852963, 1323.729793],
       [1285.542171, 1281.236155, 1275.901616],
       [1343.14893 ,         nan, 1330.419743],
       [1287.110989, 1290.582636, 1301.400939],
       [1490.183903, 1489.269307, 1507.705215],
       [1397.477907,         nan, 1413.641134],
       [1409.592878, 1406.415937, 1374.768872],
       [        nan,         nan,         nan],
       [        nan, 1307.726654, 1322.044339],
       [        nan, 1283.523501, 1250.438056],
       [1375.988165, 1395.746145, 1382.351749],
       [1422.062139, 1418.902881, 1414.620973],
       [        nan, 1509.131218, 1440.690114],
       [1280.614143, 1264.430907,         nan],
       [1456.708056, 1619.965046, 1536.395687],
       [1363.709677, 1392.174508, 1374.226755],
       [1573.006294, 1406.529655, 1617.606567],
       [        nan, 1451.901795,         nan],
       [1305.304579, 1296.522271,         nan],
       [1530.525521,         nan, 1524.87142 ],
       [1388.183309,         nan, 1611.3